In [0]:
%pip install smbprotocol

In [0]:
dbutils.library.restartPython()

In [0]:
import uuid
from smbprotocol.connection import Connection
from smbprotocol.session import Session
from smbprotocol.tree import TreeConnect
from smbprotocol.open import Open, FilePipePrinterAccessMask, CreateDisposition, ShareAccess
from smbprotocol.open import FileAttributes, CreateOptions, ImpersonationLevel
import tempfile
from datetime import datetime

# Retrieve secrets for SMB connection
server = dbutils.secrets.get(scope="goc_secrets", key="IPServerStatusPage")
share = dbutils.secrets.get(scope="goc_secrets", key="ShareStatusPage")
username = dbutils.secrets.get(scope="goc_secrets", key="userWindows")
password = dbutils.secrets.get(scope="goc_secrets", key="passWindows")

# Read data from the specified table and filter based on the latest iso_week_num and conditions
basedStatusPage = spark.read.table("ds_goc_bronze_dev.ds_goc_gold_dev.curated_statuspage") \
    .filter("""
        iso_week_num = (select max(iso_week_num) from ds_goc_bronze_dev.ds_goc_gold_dev.curated_statuspage)
        AND (subscription_status = 'Removed' OR client_status = 'New')
    """) \
    .selectExpr("Email", "FirstName", "LastName", "Notification",
                "CASE WHEN subscription_status = 'Removed' THEN NULL ELSE productName END AS productName")
display(basedStatusPage)
# Get the latest iso_week_num and year for file naming
latest_week_year = spark.sql("""
    SELECT max(iso_week_num) as iso_week_num
    FROM ds_goc_bronze_dev.ds_goc_gold_dev.curated_statuspage
    ORDER BY iso_week_num DESC
    LIMIT 1
""").collect()[0]
iso_week_num = latest_week_year['iso_week_num'] 
print(iso_week_num)

# Write DataFrame to a temporary CSV file
with tempfile.NamedTemporaryFile(delete=False, suffix=".csv") as tmp_file:
    tmp_file_path = tmp_file.name
    basedStatusPage.coalesce(1).write.csv(tmp_file_path, header=True, mode="overwrite")

# Read the content of the temporary CSV file
csv_file_path = tmp_file_path + "/part-00000*.csv"
csv_content = spark.read.text(csv_file_path).collect()
csv_content = "\n".join([row.value for row in csv_content])

try:
    # Generate a valid UUID for the SMB connection
    client_guid = uuid.uuid4()

    # Establish SMB connection
    conn = Connection(guid=client_guid, server_name=server, port=445)
    conn.connect()

    # Create a session with authentication
    session = Session(conn, username, password)
    session.connect()

    # Connect to the shared folder
    tree = TreeConnect(session, f"\\\\{server}\\{share}")
    tree.connect()

    # Get the current date in the format YYYYMMDD
    current_date = datetime.now().strftime("%Y%m%d")

    # Create a file named userSubscriptionsDelta_<current_date>.csv on the remote server
    file_name = f"userSubscriptionsDelta_{current_date}.csv"
    # file_name = f"userSubscriptionsDelta_{iso_week_num}.csv"
    file = Open(tree, file_name)
    file.create(
        create_disposition=CreateDisposition.FILE_OVERWRITE_IF,
        desired_access=FilePipePrinterAccessMask.GENERIC_WRITE,
        share_access=ShareAccess.FILE_SHARE_WRITE,
        impersonation_level=ImpersonationLevel.Impersonation,
        file_attributes=FileAttributes.FILE_ATTRIBUTE_NORMAL,
        create_options=CreateOptions.FILE_NON_DIRECTORY_FILE
    )
    
    # Write the CSV content to the file
    file.write(csv_content.encode(), 0)
    file.close()

    # Close connections
    tree.disconnect()
    session.disconnect()
    conn.disconnect()

    print("File transferred successfully!")

except Exception as e:
    print(f"Error: {e}")

In [0]:
import uuid
from smbprotocol.connection import Connection
from smbprotocol.session import Session
from smbprotocol.tree import TreeConnect
from smbprotocol.open import Open, FilePipePrinterAccessMask, CreateDisposition, ShareAccess
from smbprotocol.open import FileAttributes, CreateOptions, ImpersonationLevel
import tempfile
from datetime import datetime

# Retrieve secrets for SMB connection
server = dbutils.secrets.get(scope="goc_secrets", key="IPServerStatusPageNew")
share = dbutils.secrets.get(scope="goc_secrets", key="ShareStatusPage")
username = dbutils.secrets.get(scope="goc_secrets", key="userWindows")
password = dbutils.secrets.get(scope="goc_secrets", key="passWindows")

# Read data from the specified table and filter based on the latest iso_week_num and conditions
basedStatusPage = spark.read.table("ds_goc_bronze_dev.ds_goc_gold_dev.curated_statuspage") \
    .filter("""
        iso_week_num = (select max(iso_week_num) from ds_goc_bronze_dev.ds_goc_gold_dev.curated_statuspage)
        AND (subscription_status = 'Removed' OR client_status = 'New')
    """) \
    .selectExpr("Email", "FirstName", "LastName", "Notification",
                "CASE WHEN subscription_status = 'Removed' THEN NULL ELSE productName END AS productName")

# Get the latest iso_week_num and year for file naming
latest_week_year = spark.sql("""
    SELECT max(iso_week_num) as iso_week_num
    FROM ds_goc_bronze_dev.ds_goc_gold_dev.curated_statuspage
    ORDER BY iso_week_num DESC
    LIMIT 1
""").collect()[0]
iso_week_num = latest_week_year['iso_week_num'] 


# Write DataFrame to a temporary CSV file
with tempfile.NamedTemporaryFile(delete=False, suffix=".csv") as tmp_file:
    tmp_file_path = tmp_file.name
    basedStatusPage.coalesce(1).write.csv(tmp_file_path, header=True, mode="overwrite")

# Read the content of the temporary CSV file
csv_file_path = tmp_file_path + "/part-00000*.csv"
csv_content = spark.read.text(csv_file_path).collect()
csv_content = "\n".join([row.value for row in csv_content])

try:
    # Generate a valid UUID for the SMB connection
    client_guid = uuid.uuid4()

    # Establish SMB connection
    conn = Connection(guid=client_guid, server_name=server, port=445)
    conn.connect()

    # Create a session with authentication
    session = Session(conn, username, password)
    session.connect()

    # Connect to the shared folder
    tree = TreeConnect(session, f"\\\\{server}\\{share}")
    tree.connect()
    # Get the current date in the format YYYYMMDD
    current_date = datetime.now().strftime("%Y%m%d")

    # Create a file named userSubscriptionsDelta_<current_date>.csv on the remote server
    file_name = f"userSubscriptionsDelta_{current_date}.csv"

    # Create a file named userSubscriptionsDelta_<year>_<iso_week_num>.csv on the remote server
    # file_name = f"userSubscriptionsDelta_{iso_week_num}.csv"

    file = Open(tree, file_name)
    file.create(
        create_disposition=CreateDisposition.FILE_OVERWRITE_IF,
        desired_access=FilePipePrinterAccessMask.GENERIC_WRITE,
        share_access=ShareAccess.FILE_SHARE_WRITE,
        impersonation_level=ImpersonationLevel.Impersonation,
        file_attributes=FileAttributes.FILE_ATTRIBUTE_NORMAL,
        create_options=CreateOptions.FILE_NON_DIRECTORY_FILE
    )
    
    # Write the CSV content to the file
    file.write(csv_content.encode(), 0)
    file.close()

    # Close connections
    tree.disconnect()
    session.disconnect()
    conn.disconnect()

    print("File transferred successfully!")

except Exception as e:
    print(f"Error: {e}")